# 04 - Pandas - Processamento e Análise de Dados
## Aula 11 - 24 - Pandas - Remodelando Dataframes de Colunas Para Linhas (stack, unstack, melt e wide_to_long)

## Professor: Orlando Oliveira dos Santos, MsC.
 - E-mail: professor.orlando.santos@gmail.com 
 - Youtube :https://www.youtube.com/channel/UCPWWbkPWFmOjXH50TQWNAYg
 - Linkedin: https://linkedin.com/in/orlandoosantos
 - Facebook: https://www.facebook.com/proforlandosantosmsc/
 - Twitter: https://twitter.com/ProfOrlandoMsC
 - Instagram: https://www.instagram.com/proforlandosantosmsc/



## Formação Acadêmica
- Mestrado em Computação Aplicada - UnB (2014 – 2016)	
- MBA, Administração Estratégica de Sistemas de Informação – FGV (2011 – 2013)
- Tecnólogo em Processamento de Dados, Análise e Desenvolvimento de Sistemas – FAETEC/SP (2000-2002)

#  Pandas - Remodelagem e tabelas dinâmicas ( stack, unstack, melt, wide_to_long)

In [1]:
import numpy as np
import pandas as pd
import os
import zipfile

diratual = os.getcwd()
print(diratual)

diretorio = os.path.join(diratual, 'content') 
diretorio


nome_arquivo_compactado = 'HIST_PAINEL_COVIDBR_18nov2020.zip'
with zipfile.ZipFile( os.path.join(diretorio, nome_arquivo_compactado), 'r') as zip_ref:
    zip_ref.extractall(diretorio)

C:\Orlando\Curso-de-Python-Para-Analise-de-Dados\04 - Pandas - Processamento e Análise de Dados


In [2]:
import pandas as pd
dfcovid = pd.read_csv(os.path.join(diretorio, 'HIST_PAINEL_COVIDBR_18nov2020.csv') , sep=';')
dfcovid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


In [3]:
filtro = (dfcovid['estado'].isna() == False ) & ( dfcovid['codmun'].isna()  )
dfcovid = dfcovid[filtro][['regiao', 'estado', 'data' , 'populacaoTCU2019',
       'casosNovos', 'obitosNovos']]

dfcovid['mes'] = dfcovid['data'].map(lambda x : x[:7])


print(dfcovid.shape)
dfcovid.head()

(7290, 7)


,regiao,estado,data,populacaoTCU2019,casosNovos,obitosNovos,mes
270,Norte,RO,2020-02-25,1777225.0,0,0,2020-02
271,Norte,RO,2020-02-26,1777225.0,0,0,2020-02
272,Norte,RO,2020-02-27,1777225.0,0,0,2020-02
273,Norte,RO,2020-02-28,1777225.0,0,0,2020-02
274,Norte,RO,2020-02-29,1777225.0,0,0,2020-02


In [4]:
dfregiaouf = dfcovid[['regiao', 'estado', 'mes',  'casosNovos', 'obitosNovos', 'populacaoTCU2019', 'data']]\
  .groupby(['regiao', 'estado', 'mes', 'data' ])\
  .agg({'casosNovos':'sum', 'obitosNovos':'sum', 'populacaoTCU2019':'max'})\
  .reset_index()
dfregiaouf.head()

,regiao,estado,mes,data,casosNovos,obitosNovos,populacaoTCU2019
0,Centro-Oeste,DF,2020-02,2020-02-25,0,0,3015268.0
1,Centro-Oeste,DF,2020-02,2020-02-26,0,0,3015268.0
2,Centro-Oeste,DF,2020-02,2020-02-27,0,0,3015268.0
3,Centro-Oeste,DF,2020-02,2020-02-28,0,0,3015268.0
4,Centro-Oeste,DF,2020-02,2020-02-29,0,0,3015268.0


In [5]:
dfregiaouf.columns

Index(['regiao', 'estado', 'mes', 'data', 'casosNovos', 'obitosNovos',
       'populacaoTCU2019'],
      dtype='object')

In [6]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum,
                      margins=True, margins_name = 'Total')
print(dfresumo.loc['Total'].sum())
dfresumo.head(30)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.024876,0.109453,0.019900,0.800995,0.044776
2020-04,0.012982,0.265614,0.124211,0.567193,0.030000
2020-05,0.012643,0.313074,0.212617,0.445906,0.015760
2020-06,0.044749,0.343857,0.126684,0.449868,0.034841
2020-07,0.109182,0.284085,0.070679,0.435875,0.100179
2020-08,0.135197,0.224901,0.056251,0.442261,0.141389
2020-09,0.150990,0.180630,0.069337,0.462629,0.136414
2020-10,0.137459,0.187045,0.058499,0.487509,0.129488


## Remodelando com  empilhamento e  desempilhamento  

O Pandas possui métodos stack()e unstack() relacionados ao pivot(), que para dinamizar para trabalhar em conjunto com MultiIndex.

- stack: “Dinamizar” um nível dos rótulos de coluna (possivelmente hierárquicos), retornando a DataFrame com um índice com um novo nível mais interno de rótulos de linha.


- unstack: (operação inversa de stack) “dinamiza” um nível do índice de linha (possivelmente hierárquico) para o eixo da coluna, produzindo um remodelado DataFramecom um novo nível mais interno de rótulos de coluna.


- Se as colunas tiverem um MultiIndex, pode-se escolher qual nível empilhar. O nível empilhado se torna o novo nível mais baixo em a MultiIndexnas colunas.


- A operação inversa de  unstack por padrão descompacta o último nível.


- Se os índices tiverem nomes, pode-se usar os nomes dos níveis em vez de especificar os números dos níveis

In [7]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'],
                       values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
dfresumo.head(28)

Região  Centro-Oeste                  Nordeste                               \
Estados           DF    GO   MS    MT       AL    BA    CE    MA   PB    PE   
Meses                                                                         
2020-02            0     0    0     0        0     0     0     0    0     0   
2020-03            3     1    1     0        1     2     7     1    0     6   
2020-04           27    28    8    11       46   102   475   183   62   559   
2020-05          140    95   11    50      396   563  2528   771  298  2242   
2020-06          417   350   56   532      609  1186  3136  1093  617  2022   
2020-07          882  1182  300  1226      515  1610  1522   965  834  1728   
2020-08         1052  1438  486   932      320  1934   741   433  639  1036   
2020-09          734  1578  441   655      185  1347   585   310  374   658   
2020-10          428  1081  289   392      166   878   359   297  277   376   
2020-11          174   448  121   227       73   439   124   165  135   272   

Região   ... Norte           Sudeste                     Sul              
Estados  ...    RO   RR   TO      ES    MG    RJ    SP    PR    RS    SC  
Meses    ...                                                              
2020-02  ...     0    0    0       0     0     0     0     0     0     0  
2020-03  ...     1    0    0       0     2    23   136     3     4     2  
2020-04  ...    15    7    3      83    80   831  2239    80    47    44  
2020-05  ...   140  109   70     521   189  4490  5240    99   173    97  
2020-06  ...   362  167  127    1044   694  4736  7148   467   390   198  
2020-07  ...   354  222  181     897  1804  3397  8234  1271  1262   761  
2020-08  ...   276   82  292     613  2566  2588  7017  1370  1559  1158  
2020-09  ...   209   61  266     387  2025  2422  5608  1195  1347   537  
2020-10  ...   100   44  158     310  1655  2113  3689   731  1015   317  
2020-11  ...    58   28   50     246   673  1338  1868   616   650   321  

[10 rows x 27 columns]

In [8]:
dfregistros =  dfresumo.stack()
dfregistros.tail()

Região           Centro-Oeste  Nordeste  Norte  Sudeste    Sul
Meses   Estados                                               
2020-11 RS                NaN       NaN    NaN      NaN  650.0
        SC                NaN       NaN    NaN      NaN  321.0
        SE                NaN      68.0    NaN      NaN    NaN
        SP                NaN       NaN    NaN   1868.0    NaN
        TO                NaN       NaN   50.0      NaN    NaN

In [9]:
dfregistros =  dfresumo.copy()
dfregistros.columns = [ x[1] for x in dfregistros.columns ]
dfregistros.tail()

,DF,GO,MS,MT,AL,BA,CE,MA,PB,PE,...,RO,RR,TO,ES,MG,RJ,SP,PR,RS,SC
Meses,,,,,,,,,,,,,,,,,,,,,
2020-07,882,1182,300,1226,515,1610,1522,965,834,1728,...,354,222,181,897,1804,3397,8234,1271,1262,761
2020-08,1052,1438,486,932,320,1934,741,433,639,1036,...,276,82,292,613,2566,2588,7017,1370,1559,1158
2020-09,734,1578,441,655,185,1347,585,310,374,658,...,209,61,266,387,2025,2422,5608,1195,1347,537
2020-10,428,1081,289,392,166,878,359,297,277,376,...,100,44,158,310,1655,2113,3689,731,1015,317
2020-11,174,448,121,227,73,439,124,165,135,272,...,58,28,50,246,673,1338,1868,616,650,321


In [10]:
dfregistros =  dfregistros.stack()
dfregistros.reset_index().tail()

,Meses,level_1,0
265,2020-11,RJ,1338
266,2020-11,SP,1868
267,2020-11,PR,616
268,2020-11,RS,650
269,2020-11,SC,321


In [11]:
dfresumo.head()

Região  Centro-Oeste               Nordeste                               ...  \
Estados           DF   GO  MS   MT       AL    BA    CE    MA   PB    PE  ...   
Meses                                                                     ...   
2020-02            0    0   0    0        0     0     0     0    0     0  ...   
2020-03            3    1   1    0        1     2     7     1    0     6  ...   
2020-04           27   28   8   11       46   102   475   183   62   559  ...   
2020-05          140   95  11   50      396   563  2528   771  298  2242  ...   
2020-06          417  350  56  532      609  1186  3136  1093  617  2022  ...   

Região  Norte           Sudeste                   Sul            
Estados    RO   RR   TO      ES   MG    RJ    SP   PR   RS   SC  
Meses                                                            
2020-02     0    0    0       0    0     0     0    0    0    0  
2020-03     1    0    0       0    2    23   136    3    4    2  
2020-04    15    7    3      83   80   831  2239   80   47   44  
2020-05   140  109   70     521  189  4490  5240   99  173   97  
2020-06   362  167  127    1044  694  4736  7148  467  390  198  

[5 rows x 27 columns]

In [12]:
dfregistros =  dfresumo.stack(1)
dfregistros.tail()

Região           Centro-Oeste  Nordeste  Norte  Sudeste    Sul
Meses   Estados                                               
2020-11 RS                NaN       NaN    NaN      NaN  650.0
        SC                NaN       NaN    NaN      NaN  321.0
        SE                NaN      68.0    NaN      NaN    NaN
        SP                NaN       NaN    NaN   1868.0    NaN
        TO                NaN       NaN   50.0      NaN    NaN

In [13]:
dfregistros =  dfresumo.stack(['Região', 'Estados']).reset_index()
dfregistros.head()

,Meses,Região,Estados,0
0,2020-02,Centro-Oeste,DF,0.0
1,2020-02,Centro-Oeste,GO,0.0
2,2020-02,Centro-Oeste,MS,0.0
3,2020-02,Centro-Oeste,MT,0.0
4,2020-02,Nordeste,AL,0.0


In [14]:
dfregistros =  dfresumo.stack([0,1]).reset_index()
dfregistros.head()

,Meses,Região,Estados,0
0,2020-02,Centro-Oeste,DF,0.0
1,2020-02,Centro-Oeste,GO,0.0
2,2020-02,Centro-Oeste,MS,0.0
3,2020-02,Centro-Oeste,MT,0.0
4,2020-02,Nordeste,AL,0.0


In [15]:
dfregistros =  dfresumo.stack(['Região', 'Estados']).unstack('Meses')
dfregistros.head()

Meses                 2020-02  2020-03  2020-04  2020-05  2020-06  2020-07  \
Região       Estados                                                         
Centro-Oeste DF           0.0      3.0     27.0    140.0    417.0    882.0   
             GO           0.0      1.0     28.0     95.0    350.0   1182.0   
             MS           0.0      1.0      8.0     11.0     56.0    300.0   
             MT           0.0      0.0     11.0     50.0    532.0   1226.0   
Nordeste     AL           0.0      1.0     46.0    396.0    609.0    515.0   

Meses                 2020-08  2020-09  2020-10  2020-11  
Região       Estados                                      
Centro-Oeste DF        1052.0    734.0    428.0    174.0  
             GO        1438.0   1578.0   1081.0    448.0  
             MS         486.0    441.0    289.0    121.0  
             MT         932.0    655.0    392.0    227.0  
Nordeste     AL         320.0    185.0    166.0     73.0

In [16]:
dfregistros =  dfresumo.stack([0,1]).unstack(0)
dfregistros.head()

Meses                 2020-02  2020-03  2020-04  2020-05  2020-06  2020-07  \
Região       Estados                                                         
Centro-Oeste DF           0.0      3.0     27.0    140.0    417.0    882.0   
             GO           0.0      1.0     28.0     95.0    350.0   1182.0   
             MS           0.0      1.0      8.0     11.0     56.0    300.0   
             MT           0.0      0.0     11.0     50.0    532.0   1226.0   
Nordeste     AL           0.0      1.0     46.0    396.0    609.0    515.0   

Meses                 2020-08  2020-09  2020-10  2020-11  
Região       Estados                                      
Centro-Oeste DF        1052.0    734.0    428.0    174.0  
             GO        1438.0   1578.0   1081.0    448.0  
             MS         486.0    441.0    289.0    121.0  
             MT         932.0    655.0    392.0    227.0  
Nordeste     AL         320.0    185.0    166.0     73.0

### O  unstack pode receber um argumento opcional fill_value, para especificar o valor dos dados vazios.

In [17]:
dfregistros =   dfresumo.stack(['Região', 'Estados']).unstack(fill_value=0)
dfregistros.tail()

Estados                 AC    AL     AM    AP     BA     CE     DF     ES  \
Meses   Região                                                              
2020-11 Centro-Oeste   0.0   0.0    0.0   0.0    0.0    0.0  174.0    0.0   
        Nordeste       0.0  73.0    0.0   0.0  439.0  124.0    0.0    0.0   
        Norte         18.0   0.0  218.0  39.0    0.0    0.0    0.0    0.0   
        Sudeste        0.0   0.0    0.0   0.0    0.0    0.0    0.0  246.0   
        Sul            0.0   0.0    0.0   0.0    0.0    0.0    0.0    0.0   

Estados                  GO     MA  ...     PR      RJ    RN    RO    RR  \
Meses   Região                      ...                                    
2020-11 Centro-Oeste  448.0    0.0  ...    0.0     0.0   0.0   0.0   0.0   
        Nordeste        0.0  165.0  ...    0.0     0.0  80.0   0.0   0.0   
        Norte           0.0    0.0  ...    0.0     0.0   0.0  58.0  28.0   
        Sudeste         0.0    0.0  ...    0.0  1338.0   0.0   0.0   0.0   
        Sul             0.0    0.0  ...  616.0     0.0   0.0   0.0   0.0   

Estados                  RS     SC    SE      SP    TO  
Meses   Região                                          
2020-11 Centro-Oeste    0.0    0.0   0.0     0.0   0.0  
        Nordeste        0.0    0.0  68.0     0.0   0.0  
        Norte           0.0    0.0   0.0     0.0  50.0  
        Sudeste         0.0    0.0   0.0  1868.0   0.0  
        Sul           650.0  321.0   0.0     0.0   0.0  

[5 rows x 27 columns]

## Remodelagem com a função  melt()

A função melt() é útil para transformar um DataFrame em um formato onde uma ou mais colunas são variáveis **identificadoras** , enquanto todas as outras colunas, consideradas variáveis **medidas** , são "não dinamizadas" para o eixo da linha, resultando apenas duas colunas não identificadoras , “Variável” e “valor”. 



In [18]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               index=['mes'], columns=['estado'])
dfresumo.head(10)

obitosNovos                                                         \
estado           AC         AL         AM        AP         BA          CE   
mes                                                                          
2020-02    0.000000   0.000000   0.000000  0.000000   0.000000    0.000000   
2020-03    0.000000   0.032258   0.096774  0.000000   0.064516    0.225806   
2020-04    0.633333   1.533333  14.066667  1.133333   3.400000   15.833333   
2020-05    4.161290  12.774194  52.483871  6.064516  18.161290   81.548387   
2020-06    7.233333  20.300000  25.700000  6.500000  39.533333  104.533333   
2020-07    5.354839  16.612903  14.354839  4.774194  51.935484   49.096774   
2020-08    2.612903  10.322581  12.290323  3.096774  62.387097   23.903226   
2020-09    1.566667   6.166667  16.900000  1.600000  44.900000   19.500000   
2020-10    1.096774   5.354839  12.483871  1.258065  28.322581   11.580645   
2020-11    0.900000   3.650000  10.900000  1.950000  21.950000    6.200000   

                                                     ...             \
estado          DF         ES         GO         MA  ...         PR   
mes                                                  ...              
2020-02   0.000000   0.000000   0.000000   0.000000  ...   0.000000   
2020-03   0.096774   0.000000   0.032258   0.032258  ...   0.096774   
2020-04   0.900000   2.766667   0.933333   6.100000  ...   2.666667   
2020-05   4.516129  16.806452   3.064516  24.870968  ...   3.193548   
2020-06  13.900000  34.800000  11.666667  36.433333  ...  15.566667   
2020-07  28.451613  28.935484  38.129032  31.129032  ...  41.000000   
2020-08  33.935484  19.774194  46.387097  13.967742  ...  44.193548   
2020-09  24.466667  12.900000  52.600000  10.333333  ...  39.833333   
2020-10  13.806452  10.000000  34.870968   9.580645  ...  23.580645   
2020-11   8.700000  12.300000  22.400000   8.250000  ...  30.800000   

                                                                           \
estado           RJ         RN         RO        RR         RS         SC   
mes                                                                         
2020-02    0.000000   0.000000   0.000000  0.000000   0.000000   0.000000   
2020-03    0.741935   0.032258   0.032258  0.000000   0.129032   0.064516   
2020-04   27.700000   1.833333   0.500000  0.233333   1.566667   1.466667   
2020-05  144.838710   8.032258   4.516129  3.516129   5.580645   3.129032   
2020-06  157.866667  24.300000  12.066667  5.566667  13.000000   6.600000   
2020-07  109.580645  23.967742  11.419355  7.161290  40.709677  24.548387   
2020-08   83.483871  15.451613   8.903226  2.645161  50.290323  37.354839   
2020-09   80.733333   4.566667   6.966667  2.033333  44.900000  17.900000   
2020-10   68.161290   5.935484   3.225806  1.419355  32.741935  10.225806   
2020-11   66.900000   4.000000   2.900000  1.400000  32.500000  16.050000   

                                          
estado          SE          SP        TO  
mes                                       
2020-02   0.000000    0.000000  0.000000  
2020-03   0.000000    4.387097  0.000000  
2020-04   0.400000   74.633333  0.100000  
2020-05   4.709677  169.032258  2.258065  
2020-06  17.266667  238.266667  4.233333  
2020-07  24.451613  265.612903  5.838710  
2020-08  13.645161  226.354839  9.419355  
2020-09   5.966667  186.933333  8.866667  
2020-10   5.483871  119.000000  5.096774  
2020-11   3.400000   93.400000  2.500000  

[10 rows x 27 columns]

In [19]:
varscolumns = [c for _ , c in  dfresumo.columns]
print(varscolumns)

dfresumo.columns = varscolumns
dfresumo.reset_index(inplace=True)
dfresumo.head()

['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']


,mes,AC,AL,AM,AP,BA,CE,DF,ES,GO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-03,0.000000,0.032258,0.096774,0.000000,0.064516,0.225806,0.096774,0.000000,0.032258,...,0.096774,0.741935,0.032258,0.032258,0.000000,0.129032,0.064516,0.000000,4.387097,0.000000
2,2020-04,0.633333,1.533333,14.066667,1.133333,3.400000,15.833333,0.900000,2.766667,0.933333,...,2.666667,27.700000,1.833333,0.500000,0.233333,1.566667,1.466667,0.400000,74.633333,0.100000
3,2020-05,4.161290,12.774194,52.483871,6.064516,18.161290,81.548387,4.516129,16.806452,3.064516,...,3.193548,144.838710,8.032258,4.516129,3.516129,5.580645,3.129032,4.709677,169.032258,2.258065
4,2020-06,7.233333,20.300000,25.700000,6.500000,39.533333,104.533333,13.900000,34.800000,11.666667,...,15.566667,157.866667,24.300000,12.066667,5.566667,13.000000,6.600000,17.266667,238.266667,4.233333


In [20]:
dfresumo.head()

,mes,AC,AL,AM,AP,BA,CE,DF,ES,GO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-03,0.000000,0.032258,0.096774,0.000000,0.064516,0.225806,0.096774,0.000000,0.032258,...,0.096774,0.741935,0.032258,0.032258,0.000000,0.129032,0.064516,0.000000,4.387097,0.000000
2,2020-04,0.633333,1.533333,14.066667,1.133333,3.400000,15.833333,0.900000,2.766667,0.933333,...,2.666667,27.700000,1.833333,0.500000,0.233333,1.566667,1.466667,0.400000,74.633333,0.100000
3,2020-05,4.161290,12.774194,52.483871,6.064516,18.161290,81.548387,4.516129,16.806452,3.064516,...,3.193548,144.838710,8.032258,4.516129,3.516129,5.580645,3.129032,4.709677,169.032258,2.258065
4,2020-06,7.233333,20.300000,25.700000,6.500000,39.533333,104.533333,13.900000,34.800000,11.666667,...,15.566667,157.866667,24.300000,12.066667,5.566667,13.000000,6.600000,17.266667,238.266667,4.233333


In [21]:
dfresumo.melt(id_vars=['mes']).head(10)

,mes,variable,value
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,0.900000


Os nomes dessas duas colunas podem ser personalizados fornecendo os parâmetros var_namee value_name.

In [22]:
dfresumo.melt(id_vars=['mes'], var_name='estado', value_name='obitos').head(10)

,mes,estado,obitos
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,0.900000


Os valores do índice original podem ser mantidos indicando o parâmetro ignore_index como False(o padrão é True), mas isso irá duplicá-los.

In [23]:
dfmelt = dfresumo.melt(id_vars=['mes'], var_name='estado', value_name='obitos',  ignore_index=False)
dfmelt.head(10)

,mes,estado,obitos
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,0.900000


In [24]:
dfmelt.loc[1].head()

,mes,estado,obitos
1,2020-03,AC,0.000000
1,2020-03,AL,0.032258
1,2020-03,AM,0.096774
1,2020-03,AP,0.000000
1,2020-03,BA,0.064516


## Remodelagem com a função wide_to_long()

Outra maneira de transformar é usar a função wide_to_long() que é menos flexível melt(), mas mais amigável.

## Remodelagem com a função wide_to_long()

Outra maneira de transformar é usar a função wide_to_long() que é menos flexível melt(), mas mais amigável.

In [25]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               index=['regiao','estado'], columns=['mes'])
varscolumns = [c for _ , c in  dfresumo.columns]
print(varscolumns)

dfresumo.columns = varscolumns
dfresumo.reset_index(inplace=True)
dfresumo.head()

['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11']


,regiao,estado,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11
0,Centro-Oeste,DF,0.0,0.096774,0.900000,4.516129,13.900000,28.451613,33.935484,24.466667,13.806452,8.70
1,Centro-Oeste,GO,0.0,0.032258,0.933333,3.064516,11.666667,38.129032,46.387097,52.600000,34.870968,22.40
2,Centro-Oeste,MS,0.0,0.032258,0.266667,0.354839,1.866667,9.677419,15.677419,14.700000,9.322581,6.05
3,Centro-Oeste,MT,0.0,0.000000,0.366667,1.612903,17.733333,39.548387,30.064516,21.833333,12.645161,11.35
4,Nordeste,AL,0.0,0.032258,1.533333,12.774194,20.300000,16.612903,10.322581,6.166667,5.354839,3.65


In [26]:
df2021 = dfresumo.copy()
df2021.columns = [ x.replace('2020','2021' ) for x in df2021.columns]

colunasmeses = [ x for x in df2021.columns if '2021' in x]
df20202021 = pd.concat([dfresumo, df2021[colunasmeses]*.10]
                       , axis=1)
df20202021.head()

,regiao,estado,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11
0,Centro-Oeste,DF,0.0,0.096774,0.900000,4.516129,13.900000,28.451613,33.935484,24.466667,...,0.0,0.009677,0.090000,0.451613,1.390000,2.845161,3.393548,2.446667,1.380645,0.870
1,Centro-Oeste,GO,0.0,0.032258,0.933333,3.064516,11.666667,38.129032,46.387097,52.600000,...,0.0,0.003226,0.093333,0.306452,1.166667,3.812903,4.638710,5.260000,3.487097,2.240
2,Centro-Oeste,MS,0.0,0.032258,0.266667,0.354839,1.866667,9.677419,15.677419,14.700000,...,0.0,0.003226,0.026667,0.035484,0.186667,0.967742,1.567742,1.470000,0.932258,0.605
3,Centro-Oeste,MT,0.0,0.000000,0.366667,1.612903,17.733333,39.548387,30.064516,21.833333,...,0.0,0.000000,0.036667,0.161290,1.773333,3.954839,3.006452,2.183333,1.264516,1.135
4,Nordeste,AL,0.0,0.032258,1.533333,12.774194,20.300000,16.612903,10.322581,6.166667,...,0.0,0.003226,0.153333,1.277419,2.030000,1.661290,1.032258,0.616667,0.535484,0.365


In [27]:
pd.wide_to_long(df20202021, stubnames=["2020",'2021'], sep='-', i=['regiao','estado'], j="Mês").reset_index()

,regiao,estado,Mês,2020,2021
0,Centro-Oeste,DF,2,0.000000,0.000000
1,Centro-Oeste,DF,3,0.096774,0.009677
2,Centro-Oeste,DF,4,0.900000,0.090000
3,Centro-Oeste,DF,5,4.516129,0.451613
4,Centro-Oeste,DF,6,13.900000,1.390000
...,...,...,...,...,...
265,Sul,SC,7,24.548387,2.454839
266,Sul,SC,8,37.354839,3.735484
267,Sul,SC,9,17.900000,1.790000
268,Sul,SC,10,10.225806,1.022581


## Combinando com estatísticas e GroupBy 

- A combinação de pivot/ stack/ unstack com GroupBy e as funções estatísticas básicas de Series e DataFrame podem produzir algumas manipulações de dados muito expressivas e rápidas.

In [28]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               columns=['regiao','estado'], index=['mes'])
dfresumo.head()

obitosNovos                                                        \
regiao  Centro-Oeste                                   Nordeste              
estado            DF         GO        MS         MT         AL         BA   
mes                                                                          
2020-02     0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
2020-03     0.096774   0.032258  0.032258   0.000000   0.032258   0.064516   
2020-04     0.900000   0.933333  0.266667   0.366667   1.533333   3.400000   
2020-05     4.516129   3.064516  0.354839   1.612903  12.774194  18.161290   
2020-06    13.900000  11.666667  1.866667  17.733333  20.300000  39.533333   

                                                      ...             \
regiao                                                ...      Norte   
estado           CE         MA         PB         PE  ...         RO   
mes                                                   ...              
2020-02    0.000000   0.000000   0.000000   0.000000  ...   0.000000   
2020-03    0.225806   0.032258   0.000000   0.193548  ...   0.032258   
2020-04   15.833333   6.100000   2.066667  18.633333  ...   0.500000   
2020-05   81.548387  24.870968   9.612903  72.322581  ...   4.516129   
2020-06  104.533333  36.433333  20.566667  67.400000  ...  12.066667   

                                                                           \
regiao                         Sudeste                                      
estado         RR        TO         ES         MG          RJ          SP   
mes                                                                         
2020-02  0.000000  0.000000   0.000000   0.000000    0.000000    0.000000   
2020-03  0.000000  0.000000   0.000000   0.064516    0.741935    4.387097   
2020-04  0.233333  0.100000   2.766667   2.666667   27.700000   74.633333   
2020-05  3.516129  2.258065  16.806452   6.096774  144.838710  169.032258   
2020-06  5.566667  4.233333  34.800000  23.133333  157.866667  238.266667   

                                         
regiao         Sul                       
estado          PR         RS        SC  
mes                                      
2020-02   0.000000   0.000000  0.000000  
2020-03   0.096774   0.129032  0.064516  
2020-04   2.666667   1.566667  1.466667  
2020-05   3.193548   5.580645  3.129032  
2020-06  15.566667  13.000000  6.600000  

[5 rows x 27 columns]

In [29]:
dfresumo.stack().head()

obitosNovos                           
regiao         Centro-Oeste Nordeste Norte Sudeste Sul
mes     estado                                        
2020-02 AC              NaN      NaN   0.0     NaN NaN
        AL              NaN      0.0   NaN     NaN NaN
        AM              NaN      NaN   0.0     NaN NaN
        AP              NaN      NaN   0.0     NaN NaN
        BA              NaN      0.0   NaN     NaN NaN

In [30]:
dfresumo.stack().sum()

             regiao      
obitosNovos  Centro-Oeste     531.959140
             Nordeste        1453.955914
             Norte            547.976882
             Sudeste         2583.375806
             Sul              539.688710
dtype: float64

In [31]:
dfresumo.stack().sum().unstack()

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
obitosNovos,531.95914,1453.955914,547.976882,2583.375806,539.68871


In [32]:
dfresumo.stack().sum().unstack().sum(1)

obitosNovos    5656.956452
dtype: float64

In [33]:
dfresumo.groupby(level=1, axis=1).sum().sum()

regiao
Centro-Oeste     531.959140
Nordeste        1453.955914
Norte            547.976882
Sudeste         2583.375806
Sul              539.688710
dtype: float64

In [34]:
for g, data in dfresumo.groupby(level=1, axis=1):
    print(g, data)

Centro-Oeste          obitosNovos                                 
regiao  Centro-Oeste                                 
estado            DF         GO         MS         MT
mes                                                  
2020-02     0.000000   0.000000   0.000000   0.000000
2020-03     0.096774   0.032258   0.032258   0.000000
2020-04     0.900000   0.933333   0.266667   0.366667
2020-05     4.516129   3.064516   0.354839   1.612903
2020-06    13.900000  11.666667   1.866667  17.733333
2020-07    28.451613  38.129032   9.677419  39.548387
2020-08    33.935484  46.387097  15.677419  30.064516
2020-09    24.466667  52.600000  14.700000  21.833333
2020-10    13.806452  34.870968   9.322581  12.645161
2020-11     8.700000  22.400000   6.050000  11.350000
Nordeste         obitosNovos                                                          \
regiao     Nordeste                                                           
estado           AL         BA          CE         MA         PB